In [1]:
!pip install evaluate transformers[torch] torchaudio wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 86.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 344.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 218.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 189.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9


In [1]:
import json
from tqdm import tqdm
from collections import Counter
from datasets import DatasetDict, concatenate_datasets

In [2]:
with open("./dataset/genres.json", "r", encoding="utf-8") as f:
    genres = json.load(f)
genres

['Bad',
 'Bassy',
 'Big Room',
 'Bounce',
 'Chill',
 'Chillstep',
 'Classic',
 'Coding',
 'Country',
 'Cro',
 'Deep House',
 'Drum and Bass',
 'Dubstep',
 'EDM',
 'Electro',
 'Electro House',
 'Emotional',
 'Epic',
 'Folk',
 'Frenchcore',
 'Glitch Hop',
 'God',
 'Groove',
 'Hands Up',
 'Hardcore',
 'Hardstyle',
 'Harp',
 'Hip Hop & Rap',
 'Historic',
 'Latino',
 'Lounge',
 'Malle',
 'Minimal',
 'Motivation',
 'Orchestra Pop',
 'Orchestral Electro',
 'OVERWERK',
 'Pop',
 'Pop mit Beat',
 'Psy',
 'Psytrance',
 'RnB',
 'Rock',
 'Synthpop',
 'Techno',
 'Tekk',
 'Trance',
 'Weihnachten']

In [3]:
preprocessed_dataset = DatasetDict.load_from_disk("./dataset/music_lib")
preprocessed_dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels', 'paths'],
        num_rows: 16378
    })
    validate: Dataset({
        features: ['input_values', 'labels', 'paths'],
        num_rows: 2342
    })
    test: Dataset({
        features: ['input_values', 'labels', 'paths'],
        num_rows: 4675
    })
})

In [5]:
max_songs_per_genre = 25

counter = { genre: 0 for genre in genres }

indices_to_train = []
indices_to_test = []

current_song = ""
keep = True
for idx, sample in tqdm(enumerate(preprocessed_dataset["train"]), desc="Processing train samples", total=len(preprocessed_dataset["train"])):
    if current_song != sample["paths"]:
        current_song = sample["paths"]
        genre = genres[sample["labels"]]
        if counter[genre] == 25:
           keep = False
        else:
            keep = True
            counter[genre] += 1
    if keep:
        indices_to_train.append(idx)
    else:
        indices_to_test.append(idx)
    
len(indices_to_train), len(indices_to_test)

Processing train samples: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16378/16378 [00:58<00:00, 278.97it/s]


(4764, 11614)

In [5]:
{ genres[k]: v for k,v in sorted(Counter(preprocessed_dataset["validate"]["labels"]).items(), key=lambda item: item[1])}

{'Harp': 5,
 'Coding': 5,
 'Folk': 5,
 'Synthpop': 5,
 'Psytrance': 5,
 'Epic': 5,
 'Pop': 5,
 'OVERWERK': 5,
 'Weihnachten': 10,
 'God': 10,
 'Orchestra Pop': 10,
 'Bassy': 10,
 'Emotional': 10,
 'Pop mit Beat': 10,
 'Historic': 10,
 'Dubstep': 15,
 'Glitch Hop': 15,
 'Orchestral Electro': 15,
 'Chillstep': 15,
 'Tekk': 15,
 'RnB': 19,
 'Minimal': 20,
 'Chill': 20,
 'Deep House': 20,
 'Latino': 20,
 'Drum and Bass': 25,
 'EDM': 25,
 'Cro': 30,
 'Big Room': 30,
 'Psy': 35,
 'Frenchcore': 35,
 'Classic': 39,
 'Malle': 45,
 'Country': 45,
 'Groove': 49,
 'Electro House': 55,
 'Hip Hop & Rap': 55,
 'Motivation': 55,
 'Lounge': 55,
 'Techno': 60,
 'Bounce': 79,
 'Hardcore': 80,
 'Rock': 83,
 'Electro': 85,
 'Hardstyle': 110,
 'Bad': 148,
 'Trance': 150,
 'Hands Up': 685}

In [7]:
balanced_dataset = DatasetDict({
    "train": preprocessed_dataset["train"].select(indices_to_train),
    "validate": preprocessed_dataset["validate"],
    "test": concatenate_datasets([preprocessed_dataset["test"], preprocessed_dataset["train"].select(indices_to_test)])
})

balanced_dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels', 'paths'],
        num_rows: 4764
    })
    validate: Dataset({
        features: ['input_values', 'labels', 'paths'],
        num_rows: 2342
    })
    test: Dataset({
        features: ['input_values', 'labels', 'paths'],
        num_rows: 16289
    })
})

In [8]:
{ k: v for k,v in sorted(Counter(balanced_dataset["train"]["labels"]).items(), key=lambda item: item[1])}

{26: 20,
 18: 20,
 43: 20,
 37: 25,
 40: 30,
 36: 40,
 17: 40,
 7: 40,
 28: 56,
 38: 65,
 21: 70,
 47: 75,
 34: 80,
 16: 80,
 1: 85,
 5: 99,
 45: 100,
 35: 105,
 20: 108,
 30: 120,
 19: 120,
 0: 123,
 3: 123,
 9: 123,
 39: 124,
 15: 124,
 13: 124,
 14: 125,
 23: 125,
 27: 125,
 42: 125,
 22: 125,
 41: 125,
 46: 125,
 24: 125,
 2: 125,
 25: 125,
 31: 125,
 4: 125,
 8: 125,
 29: 125,
 11: 125,
 6: 125,
 10: 125,
 32: 125,
 33: 125,
 44: 125,
 12: 125}

In [9]:
balanced_dataset.save_to_disk("./dataset/music_lib_balanced")

Saving the dataset (0/3 shards):   0%|          | 0/4764 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/2342 [00:00<?, ? examples/s]

Saving the dataset (0/9 shards):   0%|          | 0/16289 [00:00<?, ? examples/s]